# Generate **dim_date table in Gold layer**


In [2]:
start_year: int = 2005
end_year: int = 2019

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 4, Finished, Available, Finished)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta
import calendar

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 3, Finished, Available, Finished)

In [4]:
# date range
start_date = datetime(start_year, 1, 1)
end_date = datetime(end_year, 12, 31)

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 6, Finished, Available, Finished)

In [5]:
date_list = []
current_date = start_date

while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)

print(f"Generated {len(date_list)} dates from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")


StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 7, Finished, Available, Finished)

Generated 5478 dates from 2005-01-01 to 2019-12-31


In [9]:
print(date_list[:5])

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 11, Finished, Available, Finished)

[datetime.datetime(2005, 1, 1, 0, 0), datetime.datetime(2005, 1, 2, 0, 0), datetime.datetime(2005, 1, 3, 0, 0), datetime.datetime(2005, 1, 4, 0, 0), datetime.datetime(2005, 1, 5, 0, 0)]


In [11]:
# date_time_df = spark.createDataFrame(
#      date_list, ["date_list"])
# display(date_time_df.limit(3))

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 13, Finished, Available, Finished)

In [6]:
def get_quarter_name(quarter):
    quarter_names = {1: "Q1", 2: "Q2", 3: "Q3", 4: "Q4"}
    return quarter_names.get(quarter, "Unknown")

def get_month_abbrev(month):
    return calendar.month_abbr[month]

def is_latest_year(year, max_year):
    return year == max_year

def is_latest_quarter(year, quarter, max_year, max_quarter):
    return year == max_year and quarter == max_quarter


StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 8, Finished, Available, Finished)

In [12]:
max_date = __builtins__.max(date_list)
max_year = max_date.year
max_quarter = (max_date.month - 1) // 3 + 1

for date_obj in date_list:
    year = date_obj.year
    month = date_obj.month
    quarter = (month - 1) // 3 + 1
    
    # Generate DateKey as YYYYMMDD integer
    date_key = int(date_obj.strftime('%Y%m%d'))
    
    date_dimension_data.append({
        'DateKey': date_key,
        'Date': date_obj.date(),
        'Month': month,
        'Year': year,
        'Quarter': quarter,
        'QuarterName': get_quarter_name(quarter),
        'MonthName': calendar.month_name[month],
        'MonthAbbrev': get_month_abbrev(month),
        'IsLatestYear': is_latest_year(year, max_year),
        'IsLatestQuarter': is_latest_quarter(year, quarter, max_year, max_quarter)
    })

print(f"Created {len(date_dimension_data)} date dimension records")


StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 14, Finished, Available, Finished)

Created 5478 date dimension records


In [13]:
date_dimension_data[:5]

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 15, Finished, Available, Finished)

[{'DateKey': 20050101,
  'Date': datetime.date(2005, 1, 1),
  'Month': 1,
  'Year': 2005,
  'Quarter': 1,
  'QuarterName': 'Q1',
  'MonthName': 'January',
  'MonthAbbrev': 'Jan',
  'IsLatestYear': False,
  'IsLatestQuarter': False},
 {'DateKey': 20050102,
  'Date': datetime.date(2005, 1, 2),
  'Month': 1,
  'Year': 2005,
  'Quarter': 1,
  'QuarterName': 'Q1',
  'MonthName': 'January',
  'MonthAbbrev': 'Jan',
  'IsLatestYear': False,
  'IsLatestQuarter': False},
 {'DateKey': 20050103,
  'Date': datetime.date(2005, 1, 3),
  'Month': 1,
  'Year': 2005,
  'Quarter': 1,
  'QuarterName': 'Q1',
  'MonthName': 'January',
  'MonthAbbrev': 'Jan',
  'IsLatestYear': False,
  'IsLatestQuarter': False},
 {'DateKey': 20050104,
  'Date': datetime.date(2005, 1, 4),
  'Month': 1,
  'Year': 2005,
  'Quarter': 1,
  'QuarterName': 'Q1',
  'MonthName': 'January',
  'MonthAbbrev': 'Jan',
  'IsLatestYear': False,
  'IsLatestQuarter': False},
 {'DateKey': 20050105,
  'Date': datetime.date(2005, 1, 5),
  'Month

In [7]:
date_schema = StructType([
    StructField("DateKey", IntegerType(), False),
    StructField("Date", DateType(), True),
    StructField("Month", IntegerType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Quarter", IntegerType(), True),
    StructField("QuarterName", StringType(), True),
    StructField("MonthName", StringType(), True),
    StructField("MonthAbbrev", StringType(), True),
    StructField("IsLatestYear", BooleanType(), True),
    StructField("IsLatestQuarter", BooleanType(), True)
])


StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 9, Finished, Available, Finished)

In [17]:
from pyspark.sql.functions import col
dim_date_df = spark.createDataFrame(date_dimension_data, schema=date_schema)
dim_date_df = dim_date_df.orderBy(col("DateKey").desc())
display(dim_date_df.limit(3))

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7065427b-7df3-46ca-8f40-f17aca92ff27)

In [19]:
duplicate_count = dim_date_df.groupBy("DateKey").count().filter(col("count") > 1).count()
print(f"Duplicate DateKeys: {duplicate_count}")

null_checks = dim_date_df.select([count(when(col(c).isNull(), c)).alias(c) for c in dim_date_df.columns])
print("Null value counts:")
null_checks.show()

dim_date_df.select(
    min("DateKey").alias("MinDateKey"),
    max("DateKey").alias("MaxDateKey"),
    count("DateKey").alias("TotalRecords")
).show()

print("Latest flags summary:")
dim_date_df.groupBy("IsLatestYear").count().show()
dim_date_df.groupBy("IsLatestQuarter").count().show()

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 21, Finished, Available, Finished)

Duplicate DateKeys: 0
Null value counts:
+-------+----+-----+----+-------+-----------+---------+-----------+------------+---------------+
|DateKey|Date|Month|Year|Quarter|QuarterName|MonthName|MonthAbbrev|IsLatestYear|IsLatestQuarter|
+-------+----+-----+----+-------+-----------+---------+-----------+------------+---------------+
|      0|   0|    0|   0|      0|          0|        0|          0|           0|              0|
+-------+----+-----+----+-------+-----------+---------+-----------+------------+---------------+

+----------+----------+------------+
|MinDateKey|MaxDateKey|TotalRecords|
+----------+----------+------------+
|  20050101|  20191231|        5478|
+----------+----------+------------+

Latest flags summary:
+------------+-----+
|IsLatestYear|count|
+------------+-----+
|       false| 5113|
|        true|  365|
+------------+-----+

+---------------+-----+
|IsLatestQuarter|count|
+---------------+-----+
|          false| 5386|
|           true|   92|
+---------------+-

In [22]:
dim_date_df.write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("dim_date")

print("dim_date table successfully created in silver_data lakehouse")

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 24, Finished, Available, Finished)

dim_date table successfully created in silver_data lakehouse


In [23]:
df = spark.sql("SELECT * FROM jp_real_estate_lh.dbo.dim_date LIMIT 10")
display(df)

StatementMeta(, b22c0456-ae8a-4e3d-a154-b05fd45ca927, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 896c391e-f151-402f-999b-2a21db69fcdf)